In [5]:
import json
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from google.cloud import storage
import io
from tqdm import tqdm
import os

def json_to_parquet_gcs(json_file, gcs_bucket, gcs_prefix, chunk_size=10000):
    # Initialize GCS client
    storage_client = storage.Client()
    bucket = storage_client.bucket(gcs_bucket)

    # Comment out the deletion code
    # # Remove all existing files in the destination folder
    # blobs = bucket.list_blobs(prefix=gcs_prefix)
    # for blob in blobs:
    #     blob.delete()
    # print(f"Deleted all existing files in {gcs_prefix}")

    # Function to read JSON in chunks
    def read_json_chunks(file, chunk_size):
        with open(file, 'r') as f:
            data = json.load(f)
            for i in range(0, len(data), chunk_size):
                yield i, data[i:i + chunk_size]

    # Get the list of existing files
    existing_files = list(bucket.list_blobs(prefix=gcs_prefix))
    existing_parts = set(int(blob.name.split('_')[-1].split('.')[0]) for blob in existing_files)

    # Process the JSON file in chunks
    for i, chunk in tqdm(read_json_chunks(json_file, chunk_size), desc="Converting to Parquet"):
        # Skip if this part has already been processed
        if i // chunk_size in existing_parts:
            print(f"Skipping part {i // chunk_size}, already processed")
            continue

        # Convert chunk to DataFrame
        df = pd.DataFrame(chunk)
        
        # Convert 'id' column to string
        if 'id' in df.columns:
            df['id'] = df['id'].astype(str)

        # Convert DataFrame to PyArrow Table
        table = pa.Table.from_pandas(df)
        
        # Write to Parquet in memory
        buf = io.BytesIO()
        pq.write_table(table, buf)
        buf.seek(0)
        
        # Upload to GCS
        blob_name = f"{gcs_prefix}/cambrian_dataset_10M_part_{i // chunk_size:05d}.parquet"
        blob = bucket.blob(blob_name)
        blob.upload_from_file(buf, content_type='application/octet-stream')
        
        print(f"Uploaded {blob_name}")

# File paths and GCS details
json_file = "/mnt/disks/storage/data/finetune_data/clean_9784k.json"
gcs_bucket = "us-central2-storage"
gcs_prefix = "tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset"

# Convert JSON to Parquet and upload to GCS
json_to_parquet_gcs(json_file, gcs_bucket, gcs_prefix)

print("Conversion and upload complete")

Converting to Parquet: 0it [00:00, ?it/s]

Converting to Parquet: 1it [01:53, 113.66s/it]

Skipping part 0, already processed
Skipping part 1, already processed
Skipping part 2, already processed
Skipping part 3, already processed
Skipping part 4, already processed
Skipping part 5, already processed
Skipping part 6, already processed
Skipping part 7, already processed
Skipping part 8, already processed
Skipping part 9, already processed
Skipping part 10, already processed
Skipping part 11, already processed
Skipping part 12, already processed
Skipping part 13, already processed
Skipping part 14, already processed
Skipping part 15, already processed
Skipping part 16, already processed
Skipping part 17, already processed
Skipping part 18, already processed
Skipping part 19, already processed
Skipping part 20, already processed
Skipping part 21, already processed
Skipping part 22, already processed
Skipping part 23, already processed
Skipping part 24, already processed
Skipping part 25, already processed
Skipping part 26, already processed
Skipping part 27, already processed
Sk

Converting to Parquet: 46it [01:54,  1.75s/it]

Uploaded tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset/cambrian_dataset_10M_part_00045.parquet
Uploaded tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset/cambrian_dataset_10M_part_00046.parquet
Uploaded tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset/cambrian_dataset_10M_part_00047.parquet
Uploaded tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset/cambrian_dataset_10M_part_00048.parquet
Uploaded tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset/cambrian_dataset_10M_part_00049.parquet
Uploaded tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset/cambrian_dataset_10M_part_00050.parquet
Uploaded tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset/cambrian_dataset_10M_part_00051.parquet
Uploaded tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset/cambrian_dataset_10M_part_00052.parquet


...

Uploaded tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset/cambrian_dataset_10M_part_00976.parquet


Converting to Parquet: 979it [05:09,  4.53it/s]

Uploaded tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset/cambrian_dataset_10M_part_00977.parquet
Uploaded tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset/cambrian_dataset_10M_part_00978.parquet


Converting to Parquet: 979it [05:21,  3.05it/s]

Conversion and upload complete


In [9]:
import os
import pyarrow.parquet as pq
from google.cloud import storage
import math

def download_parquet_files(gcs_bucket, gcs_prefix, local_folder):
    """Download Parquet files if they don't exist locally."""
    if not os.path.exists(local_folder):
        os.makedirs(local_folder)
    
    storage_client = storage.Client()
    bucket = storage_client.bucket(gcs_bucket)
    blobs = bucket.list_blobs(prefix=gcs_prefix)
    
    for blob in blobs:
        if blob.name.endswith('.parquet'):
            local_file_path = os.path.join(local_folder, os.path.basename(blob.name))
            if not os.path.exists(local_file_path):
                print(f"Downloading {blob.name}...")
                blob.download_to_filename(local_file_path)

def get_sample_by_id(sample_id, local_folder, samples_per_file=10000):
    """Read a sample based on its sample_id."""
    file_index = sample_id // samples_per_file
    file_name = f"cambrian_dataset_10M_part_{file_index:05d}.parquet"
    file_path = os.path.join(local_folder, file_name)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Parquet file {file_name} not found in {local_folder}")
    
    table = pq.read_table(file_path)
    df = table.to_pandas()
    
    row_index = sample_id % samples_per_file
    if row_index >= len(df):
        raise ValueError(f"Sample ID {sample_id} not found in file {file_name}")
    
    return df.iloc[row_index].to_dict()

def count_total_samples(local_folder):
    """Count the total number of samples across all Parquet files."""
    total_samples = 0
    for file_name in os.listdir(local_folder):
        if file_name.endswith('.parquet'):
            file_path = os.path.join(local_folder, file_name)
            table = pq.read_table(file_path)
            total_samples += table.num_rows
    return total_samples

# Configuration
gcs_bucket = "us-central2-storage"
gcs_prefix = "tensorflow_datasets/tensorflow_datasets/downloads/manual_cambrian_dataset"
local_folder = "/home/austinwang/manual_cambrian_dataset"

# Check if Parquet files exist locally, if not, download them
download_parquet_files(gcs_bucket, gcs_prefix, local_folder)

# Example usage

# Read a sample
sample_id = 14879
sample = get_sample_by_id(sample_id, local_folder)
print(f"Sample {sample_id}:")
print(sample)

# Count total samples
total_samples = count_total_samples(local_folder)
print(f"\nTotal number of samples: {total_samples}")

Sample 14879:
{'id': '000000441407', 'image': 'coco/train2017/000000441407.jpg', 'conversations': array([{'from': 'human', 'value': 'What color is the horse in the image?\n<image>'},
       {'from': 'gpt', 'value': 'The horse in the image is white.'},
       {'from': 'human', 'value': 'What is the horse doing in the field?'},
       {'from': 'gpt', 'value': 'The horse is standing and grazing on the grass in the field.'},
       {'from': 'human', 'value': 'What kind of equipment is the horse wearing?'},
       {'from': 'gpt', 'value': "The horse is wearing a bridle (reigns), which is used to control and guide the horse's movements."},
       {'from': 'human', 'value': 'Can you explain the purpose of a bridle and how it helps in controlling the horse?'},
       {'from': 'gpt', 'value': "A bridle is an essential piece of equipment used in horse riding and comprises several components, such as the headstall, bit, and reins. The primary purpose of a bridle is to facilitate communication and